In [1]:
!pip install torch torchvision torchaudio

     |████████████████████████████████| 1.9MB 17.0MB/s 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms